In [1]:
#!pip3 install --upgrade pip
# !pip install opencv-python
# !pip install pyperclip
# !pip install keyboard
# !pip install PySimpleGUI
# !pip install japanize_matplotlib 
# !pip install pyautogui
# !pip install pywinctl
# !pip install pygetwindow
# !pip install mojimoji
# !pip install scipy
# !pip install seaborn
# !pip install LinearRegression
#!pip install openpyxl
#!pip3 install charset-normalizer


In [2]:
import os
import pandas as pd
import openpyxl
import datetime
import pathlib
import shutil
import subprocess
import platform
from time import sleep
import pyperclip
import pickle
import glob
from openpyxl.styles import PatternFill
from openpyxl.styles.borders import Border, Side
import matplotlib.pyplot as plt
import PySimpleGUI as sg
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import japanize_matplotlib   
import pyautogui as pag 
import math    
import pywinctl as pwc
import re
from PIL import Image
import PySimpleGUI as sg
import io
import mojimoji
from scipy import stats
from scipy.stats import chi2
import numpy as np
import itertools
from charset_normalizer import detect
#非表示の時の読み込み
def ast_btn(x):
    wb = openpyxl.load_workbook(x)
    ws = wb['抽出']
    for i in range(1,ws.max_column):
        if ws.cell(2, i).value=='*':
            b=ws.cell(1,i).column_letter
            ws.column_dimensions[b].hidden= True
    wb.save(x)
#idを1ずつ増やしていくジェネレータ
def generator(n):
    while True:
        n=n+1
        yield f'd{n}'
#前回のエクセルから一番大きいidを持ってきて、そこからidを増やしていく
def id_gen(lastqx_df):
    if lastqx_df.loc[69:,'ファイル名'].isnull().all():
        xd=generator(0)
    else:
        lastqx_df.loc[69:,'ファイル名'] = lastqx_df.loc[69:,'ファイル名'].str.replace('d','')
        max_id=lastqx_df.loc[69:,['ファイル名']].astype(float).max()
        max_id=int(max_id.iloc[-1])
        xd=generator(max_id)
    return xd
#新しいtxtの作成　元々あったら作らない
def new_txt(x):
    txt_path=os.path.dirname(x)
    txt_name='id_list'
    txt_path_join=os.path.join(txt_path,txt_name)
    txt_file=pathlib.Path(txt_path_join)
    if txt_file.exists():
        pass
    else:
        f = open(txt_path_join,'w')
        f.write('')
        f.close()
    return txt_path_join
#インデックスシートへのid,日付,カラム数の書き込みリスト作成
def id_sheet(w_list,id):
    dt_now = datetime.datetime.now()
    dt_now = str(dt_now)
    w_list = [f'\n{id}',' ',f'{dt_now}\n']
    return w_list
#カラム,条件の書き込みリストへの追加
def f_write(w_list,x,y):
    w_list.extend([x,':',y,' '])
    return w_list
#indexのpklへの出力とqueryへのid付け
def output(id,m_ind,index,index_path,x,len_column):
    pkl_name=id
    pkl_path_join=os.path.join(index_path,'output',pkl_name)
    len_m_ind=len(m_ind)
    m_ind.to_pickle(pkl_path_join)
    wb = openpyxl.load_workbook(x)
    ws = wb['抽出']
    ファイル名_col=search_column(ws, 'ファイル名','column',1,ws.max_column)
    n_data_col=search_column(ws, 'データ数','column',1,ws.max_column)
    end_f_col=search_column(ws, 'end_f','column',1,ws.max_column)
    hilight(ws, [index+2], 'c1ffc1', list(range(len_column+1,end_f_col)))#lencolumns
    hilight(ws, [index+2], '66cdaa', list(range(len_column-1)))
    side1 = Side(style='thin', color='000000')
    border1 = Border(top = side1, bottom = side1, left = side1, right = side1)
    #for r in range(1):
    for col in list(range(1,end_f_col)):
        ws.cell(row = index+2, column = col).border = border1
    ws.cell(index+2,ファイル名_col).value=id
    ws.cell(index+2,n_data_col).value=len_m_ind
    wb.save(x)
#エクセル色付け
def hilight(sheet, row, color, len_columns):
    for lists in row:
        for row in sheet.iter_rows():
            for cell in row:
                if cell.row == lists:
                    if cell.column in len_columns:
                        cell.fill = PatternFill(fgColor=f'{color}', fill_type = "solid")
#ind_fに和積なしメイン
def main_query(qqdx_df,qdf,index,f,xd,index_path,x,len_column,ws):
    w_list=[]
    id = next(xd)
    w_list = id_sheet(w_list,id)
    w_list,m_ind = que_ry(w_list,qqdx_df,qdf,index,len_column,ws)
    output(id,m_ind,index,index_path,x,len_column)
    f.writelines(w_list)
    return f
#ind_fに和積入力なしパターン
def que_ry(w_list,qqdx_df,qdf,index,len_column,ws):
    ind=[]
    end_f_col=search_column(ws, 'end_f','column',1,ws.max_column)-1
    for column,value in qqdx_df.iloc[index,len_column:end_f_col].dropna().items():
        value=mojimoji.zen_to_han(str(value))#全角半角
        value=value.replace('≧','>=')
        value=value.replace('≦','<=')
        if '<' in value or '>' in value or '>=' in value or '<=' in value or '!=' in value:
            #比較演算子ありパターン
            ind,w_list=ind_com(qdf,column,value,w_list,ind)
        else:
            #比較演算子なしパターン
            ind,w_list=ind_non(qdf,column,value,w_list,ind)
    #インデックスリストの重複部分の抽出
    m_ind=dup(ind)
    return w_list,m_ind
#セル内　比較あり和積なし
def ind_com(qdf,column,value,w_list,ind):
    qdf=qdf.query(f'{column}{value}')
    q_ind=qdf.index.values
    ind.append(q_ind)
    w_list = f_write(w_list,column,value)
    return ind,w_list
#セル内　比較なし
def ind_non(qdf,column,value,w_list,ind):
    value=value.replace('=','')
    value=value.split('\n')
    w_list.extend([column,':'])
    for i in value:
        try:
            i=float(i)
        except ValueError:
            pass    
        qdf=qdf.replace(i,'RP')
        i=str(i)
        w_list.extend([i,' '])
    qdf=qdf.query(f'{column}=="RP"')
    q_ind=qdf.index.values
    ind.append(q_ind)
    return ind,w_list
#重複部分抽出
def dup(ind):
    m_ind=ind[0]
    for i in ind:
        m_ind = set(m_ind) & set(i)
    m_ind=pd.Series(data=list(m_ind))
    return m_ind
#和積の中の処理
def ms_query(index_path,qx1,qx2):
    op_path=os.path.join(index_path,'output')
    opw_path=os.path.join(op_path,'*')
    m_ind_list=glob.glob(opw_path)
    for i in m_ind_list:
        i=os.path.basename(i)
        n=i.replace('d','')
        if  qx1==n:
            dx1_path=os.path.join(op_path,i)
            dx1=pd.read_pickle(dx1_path)
        elif qx2==n:
            dx2_path=os.path.join(op_path,i)
            dx2=pd.read_pickle(dx2_path)
        else:
            pass
    dx1=list(dx1)
    dx2=list(dx2)
    return dx1,dx2
#ind_fに積
def mulitique(qx1,qx2,index_path):
    mul_w_list=[]
    dx1,dx2=ms_query(index_path,qx1,qx2)
    m_ind = list(set(dx1) & set(dx2))
    m_ind=pd.Series(data=m_ind)
    mul_w_list=[qx1,'*',qx2]
    return m_ind,mul_w_list
#メイン積
def main_multique(waseki,index,f,xd,index_path,x,len_column):
    w_list=[]
    waseki=waseki.split('*')
    qx1,qx2=waseki
    qx1=qx1.replace('d','')
    qx2=qx2.replace('d','')
    id = next(xd)
    w_list = id_sheet(w_list,id)
    m_ind,mul_w_list=mulitique(qx1,qx2,index_path)
    w_list.extend(mul_w_list)
    output(id,m_ind,index,index_path,x,len_column)
    f.writelines(w_list)
    return f
#ind_fに和
def sumque(qx1,qx2,index_path):
    sum_w_list=[]
    dx1,dx2=ms_query(index_path,qx1,qx2)
    d_ind = list(set(dx1) & set(dx2))
    dx1.extend(dx2)
    m_ind=list(set(dx1)-set(d_ind))
    m_ind=m_ind+d_ind
    m_ind=pd.Series(data=m_ind)
    sum_w_list=[qx1,'+',qx2]
    return m_ind,sum_w_list
#メイン和
def main_sumpque(waseki,index,f,xd,index_path,x,len_column):
    w_list=[]
    waseki=waseki.split('+')
    qx1,qx2=waseki
    qx1=qx1.replace('d','')
    qx2=qx2.replace('d','')
    id = next(xd)
    w_list = id_sheet(w_list,id)
    m_ind,sum_w_list=sumque(qx1,qx2,index_path)
    w_list.extend(sum_w_list)
    output(id,m_ind,index,index_path,x,len_column)
    f.writelines(w_list)     
    return f
#バイナリファイルの作成
def btn_click(df,x,project,xd,len_column):
    #前回のqueryxlを入れる
    txt_path_join=new_txt(x)
    f = open(txt_path_join, 'a')
    qdf = df.copy()
    #queryxlの読み込み,コピー
    qqx_df = pd.read_excel(x,sheet_name='抽出')
    qqdx_df = qqx_df.copy()
    #クエリエクセルから*の部分を削除する(ログに残さないために)
    dir_name=os.getcwd()
    index_path=os.path.join(dir_name,project)
    wb = openpyxl.load_workbook(x)
    ws = wb['抽出']
    end_f_col=search_column(ws, 'end_f','column',1,ws.max_column)-1
    ind_f_col=search_column(ws, '和＋積','column',1,ws.max_column)-1
    for index,row in qqdx_df[qqdx_df['ファイル名'].isnull()].iloc[69:,:end_f_col].dropna(how='all').iterrows():
        if str(row[ind_f_col])=='nan':
            f=main_query(qqdx_df,qdf,index,f,xd,index_path,x,len_column,ws)
        else:
            #waseki=str(row[ind_f_col])
            waseki=mojimoji.zen_to_han(str(row[ind_f_col]))
            if '*' in waseki:
                #積パターン
                f=main_multique(waseki,index,f,xd,index_path,x,len_column)
            else:
                #和パターン
                f=main_sumpque(waseki,index,f,xd,index_path,x,len_column)
    edit_fig_sheet(x)
    f.close()
#行ごとに書き込む
def write_list_2row(ws, l_2d, start_row, start_col):
    for y, row_data in enumerate(l_2d):
        for x, cell_data in enumerate(row_data):
            ws.cell(row=start_row + y, column=start_col + x, value=l_2d[y][x])
#データの整形、データタイプ、データ数、欠損値数のカウントの記入、1回目
def clean_count(ws,df):
    data_row=search_column(ws, 'データタイプ','row', ws.max_row, 1)
    ds_list=[]
    typ_list=[]
    nval_count=[]
    cval_count=[]
    for column in list(df.columns):
        try:
            df[column]=df[column].astype(float)
            typ_list.append('数値')
            nval_count.append(df[column].count())
            cval_count.append(0)
        except ValueError:
            df[column]=df[column].astype(str).apply(mojimoji.zen_to_han)
            df[column].replace('nan', float('nan'))
            nval=pd.to_numeric(df[column], errors="coerce").notna().sum()
            if nval==0:
                typ_list.append('文字')
            else:
                typ_list.append('数値・文字')
            nval_count.append(nval)
            cval_count.append(pd.to_numeric(df[column].dropna(), errors="coerce").isna().sum())
    ds_list.append(typ_list)
    ds_list.append(list(df.count()))
    ds_list.append(nval_count)
    ds_list.append(cval_count)
    ds_list.append(list(df.isnull().sum()))
    write_list_2row(ws, ds_list, data_row, 2)
    return df,typ_list
#要約統計量の記入
def sta(ws,df):
    dsdf=df.describe()
    dsdf=dsdf.drop('count').round(2)
    for column,values in dsdf.items():
        col_n=search_column(ws, column, 'column', 1, ws.max_column)
        row_n=search_column(ws, '平均値', 'row', ws.max_row, 1)
        values=list(values)
        for y,data in enumerate(values):
            ws.cell(row_n+y,col_n,value=data)
#メモリ上でのpng貼り付け
def xl_wr(fig,ws,cell_loc):
    img_data = io.BytesIO()
    fig.savefig(img_data, format='png')
    img = openpyxl.drawing.image.Image(img_data)
    ws.add_image(img,cell_loc)
    plt.clf()
    plt.close()
#グラフ作成
def his_sheet0(df,typ_list,ws):
    keys=list(df.columns)
    dict_keys=dict(zip(keys, typ_list))
    for num,key in enumerate(keys):
        if num==0:
            column_num=1
            row_num=28
        elif num%5==0:
            column_num=1+(num//5)*9
            row_num=28
        else:
            row_num=27*(num%5)+28
        cell_loc=ws.cell(row=row_num,column=column_num).coordinate
        df=df.dropna(how='all',axis=1)
        if key in df.dropna(how='all',axis=1).columns:
            fig,ax = plt.subplots()
            plt.title(key,fontsize=20)
            if dict_keys[key]=='文字':
                if len(df[key].unique())<68:
                    ax.hist(df[key],bins=len(df[key].unique()),histtype="bar",edgecolor="black")
                    labels = ax.get_xticklabels()
                    plt.setp(labels, rotation=-90, fontsize=7)
                    xl_wr(fig,ws,cell_loc)
                else :
                    ax.hist(df[key],histtype="bar",edgecolor="black")
                    labels = ax.get_xticklabels()
                    plt.setp(labels, rotation=-90, fontsize=10)
                    xl_wr(fig,ws,cell_loc)
            else:
                if len(df[key].unique())<10:
                    ax.hist(df[key],bins=len(df[key].unique()),histtype="bar",edgecolor="black")
                    ax.set_xticks(list(df[key].unique()))
                    xl_wr(fig,ws,cell_loc)
                else:
                    ax.hist(df[key],histtype="bar",edgecolor="black")
                    xl_wr(fig,ws,cell_loc)
        else:
            pass
#sheet0への記入、データ整形
def sheet0_write(Excel_path,df,csv_path):
    wb=openpyxl.load_workbook(Excel_path)
    ws=wb['前処理']
    df,typ_list=clean_count(ws,df)
    sta(ws,df)
    his_sheet0(df,typ_list,ws)
    wb.save(Excel_path)
    wb.close()
    df.to_csv(csv_path,encoding='cp932',)

#二回目、データタイプ削除欄の処理、外れ値処理
#データタイプ選択、タイプの書き込み
def type_change(sh0_df,df,ws,end_t_col):
    d_type_row=search_column(ws, 'データタイプ', 'row', ws.max_row, 1)
    for column,value in sh0_df.loc['数値':'文字',:'end_t'].drop('end_t',axis=1).astype(str).items():#nanの確認のためのstr
        if value[0]=='*' and value[1]=='nan':
            df[column]=pd.to_numeric(df[column], errors="coerce")
            # t_col=search_column(ws, column, 'column', d_type_row, ws.max_column)
            # ws.cell(d_type_row,t_col,value='数値')
        elif value[1]=='*' and value[0]=='nan':
            n_index=list(pd.to_numeric(df[column], errors="coerce").dropna().index)
            df[column].loc[n_index]=float('nan')
            # t_col=search_column(ws, column, 'column', d_type_row, ws.max_column)
            # ws.cell(d_type_row,t_col,value='文字')
        else:
            pass
    return df
#外れ値処理、削除数の書き込み
def out_del(sh0_df,df,ws,end_t_col):
    del_c_l=[]
    for column,value in sh0_df.loc['下限':'上限',:'end_t'].drop('end_t',axis=1).astype(str).items():
        try :
            df[column].astype(float)
            if value[0]=='nan' and value[1]=='nan':
                del_c_l.append(0)
            elif value[0]=='nan':
                value[1]=float(value[1])
                del_c=df[column][(df[column]>value[1])].count()
                df[column][(df[column]>value[1])]=float('nan')
                del_c_l.append(del_c)
            elif value[1]=='nan':
                value[0]=float(value[0])
                del_c=df[column][(df[column]<value[0])].count()
                df[column][(df[column]<value[0])]=float('nan')
                del_c_l.append(del_c)
            else:
                value=value.astype(float)
                del_c=df[column][(df[column]<value[0])|(df[column]>value[1])].count()
                df[column][(df[column]<value[0])|(df[column]>value[1])]=float('nan')
                del_c_l.append(del_c)
        except ValueError:
            del_c_l.append(0) 
    del_row=search_column(ws, '削除数', 'row', ws.max_row, 1)
    #t_col=search_column(ws, '削除数','column', 1, ws.max_column)+1
    for x, data in enumerate(list(del_c_l)):
        ws.cell(row=del_row,column=2+x, value=data)
    return df
# def rewrite(df,ws):
#     data_row=search_column(ws, 'データ数','row', ws.max_row, 1)
#     ds_list=[]
#     typ_list=[]
#     nval_count=[]
#     cval_count=[]
#     for column in list(df.columns):
#         try:
#             df[column]=df[column].astype(float)
#             typ_list.append('数値')
#             nval_count.append(df[column].count())
#             cval_count.append(0)
#         except ValueError:
#             df[column]=df[column].astype(str).apply(mojimoji.zen_to_han)
#             df[column].replace('nan', float('nan'))
#             nval=pd.to_numeric(df[column], errors="coerce").notna().sum()
#             if nval==0:
#                 typ_list.append('文字')
#             else:
#                 typ_list.append('数値・文字')
#             nval_count.append(nval)
#             cval_count.append(pd.to_numeric(df[column].dropna(), errors="coerce").isna().sum())
#     ds_list.append(list(df.count()))
#     ds_list.append(nval_count)
#     ds_list.append(cval_count)
#     ds_list.append(list(df.isnull().sum()))
#     write_list_2row(ws, ds_list, data_row, 2)#st_col)

def sheet0_rewrite(Excel_path,csv_path):
    df=pd.read_csv(csv_path,encoding='cp932',index_col=0)
    sh0_df=pd.read_excel(Excel_path,sheet_name='前処理', index_col=0)
    wb=openpyxl.load_workbook(Excel_path)
    ws=wb['前処理']
    ws._images = []
    end_t_col=search_column(ws, 'end_t', 'column', 1, ws.max_column)
    df=type_change(sh0_df,df,ws,end_t_col)
    df=out_del(sh0_df,df,ws,end_t_col)
    df,typ_list=clean_count(ws,df)
    his_sheet0(df,typ_list,ws)
    sta(ws,df)
    wb.save(Excel_path)
    wb.close()
    return df
#フレーム作成
#テキスト　辞書、いくつでも
def make_txt(txt_list):
    t = ''
    t_keys = list(txt_list.keys())
    t_values = list(txt_list.values())
    for r in range(len(txt_list)):
        t += f"[sg.Text('{t_values[r]}', key='{t_keys[r]}', font=('MS 明朝', 10, 'bold'), text_color = '#000000', background_color = '#ffffff', relief = sg.RELIEF_GROOVE),],".format(str(r))
    t = '[' + t[:-1] + ']'
    framet = eval(t)
    return framet
#inputテキスト　辞書、いくつでも
def make_ipt(ipt_list):
    i = ''
    i_keys = list(ipt_list.keys())
    i_values = list(ipt_list.values())
    for r in range(len(ipt_list)):
        i += f"[sg.InputText('{i_values[r]}', key='{i_keys[r]}', font=('MS 明朝', 10, 'bold'),  enable_events=True,),],".format(str(r))
    i = '[' + i[:-1] + ']'
    framei = eval(i)
    return framei
#参照ボタンと表示欄　辞書、表示欄のkeyとvalue、参照ボタンのvalue(keyは使用されない)
def make_fb(fb_list):
    f_keys = list(fb_list.keys())
    f_values = list(fb_list.values())
    f = f"[sg.Push(), sg.InputText('{f_values[0]}', readonly = True, key='{f_keys[0]}', font=('MS 明朝', 10, 'bold'), size = (30,20), enable_events = True,),  sg.FileBrowse(button_text = '{f_values[1]}', font = ('MS 明朝', 10, 'bold'), file_types = (('CSVファイル', '*.csv'), ('Excell ファイル', '*.xlsx'),)), sg.Push()],"
    f = '[' + f[:-1] + ']'
    framefb = eval(f)
    return framefb
#リストボックス　リスト、リストの中身が表示される
def make_list(list_list, type):
    l = f"[sg.Listbox({list_list}, key = '-projects-', font=('MS 明朝', 10, 'bold'), select_mode=sg.LISTBOX_SELECT_MODE_{type}, size = (20,10)),],"
    framel = eval(l)
    return framel
#ボタン　リスト、いくつでも
def make_btn(btn_list):
    b = ''
    for r in range(len(btn_list)):
        b += "[sg.Push(), sg.Button(btn_list[{}], size = (18,1), font = ('MS 明朝', 8 , 'bold')), sg.Push()],".format(str(r))
    b = '[' + b[:-1] + ']'
    frameb = eval(b)
    return frameb
#Tabフレーム作成
def make_tab(images_path, images_name):
    tab = ''
    for r in range(len(images_path)):
        img = eval(images_path[r])
        if os.path.basename(os.path.dirname(img)) == '散布図':
            resize_raito = 0.5
        else:
            resize_raito = 0.5
        img = load_bytesImg(img, size_ratio = resize_raito)
        image_name = str(images_name[r])
        image_name = image_name.replace('\n', '')
        tab += f"[sg.Tab('{image_name}', [[sg.Image(data = {img})]])],"#.format(str(r))
    tab = '[' + tab[:-1] + ']'
    frame_tab = eval(tab)
    return frame_tab
#フレーム表示
#プロジェクトの決定をするフレーム
def main():
    os_name = platform.system()
    scr_w, scr_h= pag.size()
    if os_name == 'Windows':
        frame_x = scr_w - 440
    elif os_name == 'Darwin':    
        frame_x = scr_w - 430
    frame_y = 0
    size_x = 400
    size_y = 300
    py_path = os.getcwd()
    projects = update_project(py_path)
    framet1 = make_txt({'-print-':'プロジェクトを選択してください'})
    framel = make_list((projects), 'SINGLE')
    frameb = make_btn(['決定', '選択解除・更新'])
    framet = make_txt({'-print-':'　　　　　　　　　　　　　　　　　　　　　'})
    if os_name == 'Windows':
        pag.moveTo(frame_x + 150, frame_y + 75)
    elif os_name == 'Darwin':
        pag.moveTo(frame_x + 150, frame_y + 100)
    frame = sg.Frame('',
        [       [sg.Push(), sg.Column(framet1), sg.Push()],     
                [sg.Push(), sg.Column(framel), sg.Column(frameb), sg.Push()],   
                [sg.Push(), sg.Column(framet), sg.Push()],     
         ],size = (size_x, size_y),           
    )
    layout = [[frame]]
    window = sg.Window('querypd', layout, resizable = True, location = (frame_x, frame_y))
    while True:
        event, values = window.read()
        if event in [None]:
            break
        elif event == '決定':
            if values['-projects-'] == ['プロジェクト新規作成']:
                window.close()
                button1_2(py_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y, projects)
            elif values['-projects-']:
                project = values['-projects-']
                project = ''.join(project)
                window.close()
                button1_1(project, py_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y)
            else:
                window['-print-'].update('プロジェクトを選択、入力してください') 
        elif event == '選択解除・更新':
            projects = update_project(py_path)
            window['-projects-'].update(projects)
            window['-print-'].update('プロジェクトリストを更新しました') 
    window.close()
#プロジェクトが新規の場合データの選択
def button1_2(py_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y, projects):
    framet1 = make_txt({'-print-':'プロジェクト名を入力、データを選択してください'})
    framei = make_ipt({'-project_name-':'プロジェクト名を入力'})
    framef = make_fb({'-original_data-':'データを選択', '':'参照'})
    frameb = make_btn(['次へ','Copy', '戻る'])
    framet = make_txt({'-print-':'　　　　　　　　　　　　　　　　　　　　　'})
    frame = sg.Frame('',
        [       [sg.Push(), sg.Column(framet1), sg.Push()], 
                [sg.Column(framei, justification = 'c')],
                [sg.Column(framef, justification = 'c')],
                [sg.Column(frameb, justification = 'r')],
                [sg.Push(), sg.Column(framet), sg.Push()],
         ],size = (size_x, size_y),                  
    )
    layout = [[frame]]
    window = sg.Window('querypd', layout, resizable = True, location = (frame_x, frame_y))
    while True:
        event, values = window.read()
        if event in [None]:
            break
        elif event == '-original_data-':
            window['-print-'].update('データが選択されました')
        elif event == '次へ':
            if values['-original_data-'] == 'データを選択' or values['-project_name-'] == 'プロジェクト名を入力':
                sg.popup_ok('入力不備があります', title = 'error', location = (frame_x + 100, frame_y + 100))
            elif values['-project_name-'] in projects:
                sg.popup_ok('すでにあるプロジェクト名です', title = 'error', location = (frame_x + 100, frame_y + 100))
            elif values['-project_name-']:
                project = values['-project_name-']
                project = ''.join(project)
                window.close()                 
                make_project(project, values['-original_data-'], py_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y)
        elif event == 'Copy':        
            t = copy_frame('project')
            window['-project_name-'].update(t)
            window['-print-'].update('コピーでプロジェクト名を取得しました')
        elif event == '戻る':
            window.close()
            main()
    window.close()
#クエリの処理
def pop_frame2(project, df , xd, project_path, book_name, os_name, frame_x, frame_y, len_column, scr_w, scr_h, size_x, size_y, data_name):
    open_query(os_name, book_name, scr_w, scr_h)
    output_dir = os.path.join(project_path, 'output')
    check_dir(output_dir)
    files = update_list(output_dir)
    sheet_lists = openpyxl.load_workbook(book_name).sheetnames
    framet1 = make_txt({'-print-':'シートを選択'})
    framel = make_list(sheet_lists, 'SINGLE')
    frameb = make_btn(['Go', 'Query', '選択解除・更新', '戻る'])
    framet = make_txt({'-print-':'　　　　　　　　　　　　　　　　　　　　　'})
    frame_l = [[sg.Push(), framet1[0][0], sg.Push()],framel[0]]
    frame = sg.Frame('',
        [       [sg.Push(), sg.Column(frame_l), sg.Column(frameb), sg.Push()],
                [sg.Push(), sg.Column(framet), sg.Push()],
         ],size = (size_x, size_y),                  
    )
    layout = [[frame]]
    window = sg.Window(project, layout, resizable = True, location = (frame_x, frame_y))
    while True:
        event, values = window.read()
        if event in [None]:
            break
        elif event == 'Go':
            close_excel('query - Excel', os_name)
            if values['-projects-'] == ['前処理']:
                df = sheet0_rewrite(book_name, data_name)
            elif values['-projects-'] == ['抽出']:
                ast_btn(book_name)
                btn_click(df, book_name, project, xd, len_column)
                files = update_list(output_dir)
            elif values['-projects-']:
                get_fig_data(output_dir, book_name, df, project, files, values['-projects-'][0], frame_x, frame_y)
            else:
                ast_btn(book_name)
                df = sheet0_rewrite(book_name, data_name)
                btn_click(df, book_name, project, xd, len_column)
                files = update_list(output_dir)
                for sheet_name in ['ヒストグラム', 'クロス集計', '箱ひげ図', '散布図', '書き出し', '相関係数', 'T検定']:
                    get_fig_data(output_dir, book_name, df, project, files, sheet_name, frame_x, frame_y)
            open_query(os_name, book_name, scr_w, scr_h)    
            window['-print-'].update('job end')
        elif event == 'Query':
            auto_fill_frame(project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, data_name)
        elif event ==  '選択解除・更新':
            window['-projects-'].update(sheet_lists)
        elif event == '戻る':
            window.close()
            main()
    window.close()
#グラフの表示
def plot_show(images_path, images_name, project_name):
    frame_tab = make_tab(images_path, images_name)
    layout = [[sg.TabGroup(frame_tab,), ]]
    window = sg.Window(project_name, layout, resizable = True, #size=(600, 500)
                       )
    while True:
        event, values = window.read()
        if event in [None]:
            break
    window.close()
#オートフィルフレーム
def auto_fill_frame(project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, data_name):
    columns_list = list(df.columns.values)
    framel = make_list(columns_list, 'SINGLE')
    frameb = make_btn(['決定', '戻る'])
    framet = make_txt({'-print-':'記入したい項目を選択して下さい'})
    frame = sg.Frame('',
        [       [sg.Push(), sg.Column(framet), sg.Push()],
                [sg.Push(), sg.Column(framel), sg.Column(frameb), sg.Push()],
         ],size = (size_x, size_y),                  
    )
    layout = [[frame]]
    window = sg.Window(project, layout, resizable = True, location = (frame_x, frame_y))
    while True:
        event, values = window.read()
        if event in [None]:
            break
        elif event == '決定':
            facter = ''.join(values['-projects-'])
            drop_list = list(dict.fromkeys(df[f'{facter}'].to_list()))
            try:
                drop_list = list(map(float, drop_list))
            except ValueError:
                auto_fill_str(facter, project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, drop_list, data_name)
            else:
                auto_fill_float(facter, project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, data_name)
            break
        elif event == '戻る':
            window.close()
            pop_frame2(project, df , xd, project_path, book_name, os_name, frame_x, frame_y, len_column, scr_w, scr_h, size_x, size_y, data_name)
    window.close()
#数字のフィル
def auto_fill_float(facter, project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, data_name):
    facter = ''.join(facter)
    framel = make_list(['≧', '≦', '＝', '＞', '＜', '!='], 'SINGLE')
    frameb = make_btn(['決定', '戻る'])
    framet = make_txt({'-print-':facter})
    frame1 = sg.Frame('',
        [       [sg.Push(), sg.Column(framet), sg.Push()],
                [sg.Push(), sg.Column(framel), sg.Column(frameb), sg.Push()],
            ],size = (size_x, size_y - 50),                  
    )
    frame2 = sg.Frame('',
        [       [sg.Text('　　　　　', key='-print-', font=('MS 明朝', 10, 'bold'), text_color = '#000000', background_color = '#ffffff', relief = sg.RELIEF_GROOVE), sg.InputText(key = '-que_num-', size=(18,1)), sg.Button('ok',  size = (9,1)), ],
            ],size = (size_x, 50), visible = False, key = 'A'                 
    )
    layout = [[frame2],
              [frame1]]
    window = sg.Window(project, layout, resizable = True, location = (frame_x, frame_y))
    facter_sign = []
    while True:
        event, values = window.read()
        if event in [None]:
            break
        elif event == '決定':
            facter_sign = []
            key = ''.join(values['-projects-'])
            facter_sign.append(key)
            window['A'].update(visible = True)
            window['-print-'].update(facter + key)
        elif event == 'ok':
            fill_keys = [''.join(facter_sign) + ''.join(values['-que_num-'])]
            close_excel('query - Excel', os_name)
            auto_fill(fill_keys, facter, '並べて', book_name)
            open_query(os_name, book_name, scr_w, scr_h)
            break
        elif event == '戻る':
            window.close()
            auto_fill_frame(project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, data_name)
    window.close()
#文字のフィル
def auto_fill_str(facter, project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, drop_list, data_name):
    facter = ''.join(facter)
    framel = make_list(drop_list, 'MULTIPLE')
    frameb = make_btn(['一ヶ所', '並べて', '戻る'])
    framet = make_txt({'-print-':facter})
    frame = sg.Frame('',
        [       [sg.Push(), sg.Column(framet), sg.Push()],
                [sg.Push(), sg.Column(framel), sg.Column(frameb), sg.Push()],
         ],size = (size_x, size_y),                  
    )
    layout = [[frame]]
    window = sg.Window(project, layout, resizable = True, location = (frame_x, frame_y))
    while True:
        event, values = window.read()
        if event in [None]:
            break
        elif event == '一ヶ所':
            close_excel('query - Excel', os_name)
            fill_keys = values['-projects-']
            auto_fill(fill_keys, facter, '一ヶ所', book_name)
            open_query(os_name, book_name, scr_w, scr_h)
            break
        elif event == '並べて':
            close_excel('query - Excel', os_name)
            fill_keys = values['-projects-']
            auto_fill(fill_keys, facter, '並べて', book_name)
            open_query(os_name, book_name, scr_w, scr_h)
            break
        elif event == '戻る':
            window.close()
            auto_fill_frame(project, frame_x, frame_y, size_x, size_y, os_name, book_name, scr_w, scr_h, df, project_path, xd, len_column, data_name)
    window.close()
#それ以外
#コピーモード開始
def copy_frame(type, os_name = 0):
    scr_w, scr_h= pag.size()
    pyperclip.copy('#')
    ttx = ''    
    if type == 'project':
        particle = '_'
    elif type == 'query':
        if os_name == 'Darwin':
            particle = '\n'
        elif os_name == 'Darwin':
            particle = ''
    while True:
        tx = copy_f(scr_w)
        ttx = ttx + tx + particle
        pyperclip.copy(ttx)
        if tx == '.':
            ttx = ttx[:-2]
            break
    if len(ttx) >= 2:
        ttx = ttx[:-1]  
    pyperclip.copy('')
    return ttx
#コピーモード
def copy_f(scr_w):
    t = pyperclip.paste()
    while True:
        sleep(.2)
        x, y = pag.position()
        if x == scr_w -1 and y == 0:
            tx = '.'
            return tx
        if t!= pyperclip.paste():
            tx = pyperclip.paste()
            tx = str(tx)
            print(tx)
            return tx
#プロジェクトが既存の場合ファイルの読み込み
def button1_1(project, py_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y):
    project_path = os.path.join(py_path, project)
    data_path = os.path.join(project_path, 'abc.csv')
    book_name = os.path.join(project_path, 'query.xlsx')
    if os.path.isfile(book_name) & os.path.isfile(data_path):         
        qx_df = pd.read_excel(book_name, sheet_name = '抽出')
        if (type(qx_df) is pd.core.frame.DataFrame):
            xd=id_gen(qx_df)
        else:
            qx_df=='0000'  
        df = pd.read_csv(data_path, encoding = 'cp932', index_col = 0)
        book = openpyxl.load_workbook(book_name)
        sheet = book['抽出']
        len_column = search_column(sheet, 'end_t', 'column', 1, sheet.max_column)
        pop_frame2(project, df , xd, project_path, book_name, os_name, frame_x, frame_y, len_column, scr_w, scr_h, size_x, size_y, data_path)
    else:   
        sg.popup_ok('プロジェクトフォルダに欠損があります', title = 'error', location = (frame_x + 100, frame_y + 100))
        main()
#データのコピー
def make_project(project, data_name, py_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y):
    project_path = os.path.join(py_path, project)
    os.makedirs(project_path, exist_ok = False)
    with open(data_name, 'rb') as f:
        c = f.read()
        result = detect(c)
    with open(data_name, encoding = result['encoding'], errors='replace') as fin:
        with open(f'{project}/abc.csv', 'w', encoding='cp932',errors='replace') as fout:
            fout.write(fin.read())
 
    make_query(project, f'{project}/abc.csv', project_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y)
#クエリのマスター作成
def make_ms_que():
    book = openpyxl.Workbook()
    sheet_names = ['書き出し', 'クロス集計', '相関係数', 'T検定', '箱ひげ図', 'ヒストグラム', '散布図', '抽出', '前処理']
    fig_list_1 = [[['コメント', 'ファイル一覧', 'To Excel', 'To CSV']], 
                    [['コメント', 'ファイル一覧', '書き出し有無']], 
                    [['コメント', 'ファイル一覧', '書き出し有無']], 
                    [['コメント', 'ファイル一覧', '書き出し有無']], 
                    [['#', '因子', '書き出し有無', '向き', '外れ値', '最小値', '最大値', '目盛間隔']], 
                    [['#', '因子', '書き出し有無', 'x最小値', 'x最大値', 'x目盛間隔', '階級幅', 'y最小値', 'y最大値', 'y目盛間隔']],
                    [['#', '因子', '書き出し有無', '最小値', '最大値', '目盛間隔']],
                    [['#', 'ファイル名', 'データ数', '和＋積']],
                    [['']]
    ]
    fig_list_2 = [[[]],
                    [[]],
                    [[]],
                    [[]],
                    [[]],
                    [[]],
                    [[]],
                    [['列非表示(*)']],
                    [[]],
    ]
    fig_list_70 = [[[]],
                    [[]],
                    [[]],
                    [['', 'タイプ']],
                    [['コメント', 'ファイル一覧', '書き出し有無']],
                    [['コメント', 'ファイル一覧', '書き出し有無']],
                    [['コメント', 'ファイル一覧', '書き出し有無']],
                    [['********ここより下に条件を記入してください*********']],
                    [[]],
    ]
    fig_list_71 = [[[]],
                    [[]],
                    [[]],
                    [[]],
                    [['save_fig']],
                    [['save_fig']],
                    [['save_fig']],
                    [[]],
                    [[]],
    ]
    for i in range(len(sheet_names)):
        make_ms_que_fig_sheet(book, sheet_names[i], fig_list_1[i], fig_list_2[i], fig_list_70[i], fig_list_71[i])
    sheet = book['前処理']
    book.move_sheet(sheet, offset = -(len(sheet_names) + 1))
    book.save('ms_query.xlsx')
#ms_queのグラフシートの書き込み
def make_ms_que_fig_sheet(book, sheet_name, fig_list_1, fig_list_2, fig_list_70, fig_list_71):
    if sheet_name == '抽出':
        sheet = book['Sheet']
        sheet.title = '抽出'
    else:
        book.create_sheet(index = 1, title = sheet_name)
    sheet = book[sheet_name]
    sheet.freeze_panes = 'A2' 
    fig_lists = []
    fig_lists_row = [1, 2, 70, 71]
    for i in [fig_list_1, fig_list_2, fig_list_70, fig_list_71]:
        fig_lists.append(i)
    for i in range(4):
        write_list(sheet, fig_lists[i], fig_lists_row[i], 1)
    point_list =[]
    if sheet_name == '抽出' or sheet_name == '箱ひげ図' or sheet_name == 'ヒストグラム' or sheet_name == '散布図' or sheet_name == 'T検定':
        point_list = [1,70]
    elif sheet_name == '書き出し' or sheet_name == '相関係数' or sheet_name == 'クロス集計':
        point_list = [1]
    len_column = sheet.max_column+1
    point_len = list(range(1, len_column))
    hilight(sheet, point_list, '66cdaa', point_len)
    if sheet_name == '抽出':
        sheet['Z1'] = 'end_t'
    if sheet_name == 'T検定':
        write_list(sheet, [['対応', '有り'], ['', '無し'], ['検定方法', 'less'], ['', 'greatar'], ['', '両側']], 71, 1)
        sheet.merge_cells(start_row = 71, end_row = 72, start_column = 1, end_column = 1)
        sheet.merge_cells(start_row = 73, end_row = 75, start_column = 1, end_column = 1)
        hilight(sheet, list(range(71, sheet.max_row + 1)), 'f9fff9', [1, 2]) 
        type_max_row = sheet.max_row + 1
        lattice_xlsx(sheet, 70, type_max_row, 1, 4)
        sheet['A99'] = 'end_t'
        hid_xlsx_row(type_max_row, 100, sheet, True)
#Excelシートへのリストの横入力
def write_list(sheet, input_list, start_row, start_col):
    for y, row_data in enumerate(input_list):
        for x, cell_data in enumerate(row_data):
            sheet.cell(row=start_row + y, column = start_col + x, value = input_list[y][x])
#Excelシートへのリストの縦入力#sheet、始まりの列、始まりの行、書き込むリスト、長さ
def make_query2(sheet, start_row, start_col, drop_list, max_drop = 0, fill_type = 0):
    if fill_type == 'Sheet_copy':
        for f in range(0, len(drop_list)):
            fill_row = sheet.max_row + 1
            sheet.cell(fill_row, start_col, value = drop_list[f])
            sheet.cell(fill_row, 1, value = drop_list[f])    
    else:
        for f in range(0, len(drop_list)):
            sheet.cell(start_row + f, start_col, value = drop_list[f])
    if len(drop_list) > max_drop:
        max_drop = len(drop_list)
    return max_drop  
#クエリの作成################項目、ファイル、セーブ
def make_query(project,data_name, project_path, os_name, frame_x, frame_y, scr_w, scr_h, size_x, size_y):
    df = pd.read_csv(data_name, encoding = 'cp932', index_col=0)
    df_columns = list(df.columns.values)#データの因子のリスト
    book_name = os.path.join(project_path, 'query.xlsx')
    if os.path.isfile(book_name):###マスターの有無を確認
        pass
    else:
        make_ms_que()
    book = openpyxl.load_workbook('ms_query.xlsx')
    float_col_lists, len_column = make_query_dxt(book['抽出'], df_columns, df)
    #plot
    sheet_names = ['箱ひげ図', 'ヒストグラム', '散布図']
    for i in range(len(sheet_names)):
        make_que_fig_sheet(book, sheet_names[i], float_col_lists)
    for sheet_name in ['相関係数', 'T検定', 'クロス集計']:
        sheet = book[sheet_name]
        sheet.cell(69, 1, value = 'end_r')
        hid_xlsx_row(2, 70, sheet, True)
    sheet = book['前処理']
    for i in [1, 17, 23]:
        write_list(sheet, [df_columns], i, 2)
    make_query2(sheet, 2, 1, ['データタイプ', 'データ数', '数値データ数', '文字データ数', '欠損値数', '平均値', '標準偏差', '最小値', '25％', '50％', '75％', '最大値', '', '', '','データタイプ選択欄', '数値', '文字', '', '', '', '外れ値処理', '下限', '上限', '削除数'])
    color_types = ['90ee90', 'ebf1de', 'daeef3', 'f2dcdb', 'e4dfec']#緑、薄緑、青、赤
    rows = [[1, 17, 23], list(range(7, 14)), [3, 4, 5, 18, 19, 24], [6, 25], [26]]
    for i in range(len(color_types)):
        hilight(sheet, rows[i], color_types[i], list(range(sheet.max_column + 1)))
    lattice_xlsx(sheet, 1, sheet.max_row + 1, 1, sheet.max_column + 1)
    sheet.cell(1, sheet.max_column + 1, value = 'end_t')
    hid_xlsx_col(sheet.max_column, sheet.max_column + 1, sheet, True)#end_t
    #保存
    book.save(book_name)
    xd = generator(0)
    sheet0_write(book_name, df, data_name)
    df = sheet0_rewrite(book_name, data_name)
    pop_frame2(project, df, xd, project_path, book_name, os_name, frame_x, frame_y, len_column, scr_w, scr_h, size_x, size_y, data_name)
#queのグラフシートの書き込み
def make_que_fig_sheet(book, sheet_name, float_col_lists):
    sheet = book[sheet_name]
    factor_col = search_column(sheet, '因子', 'column', 1, sheet.max_column) #書きこむ列を検索
    max_drop = make_query2(sheet, 2, factor_col, float_col_lists, 0)#書き込み
    hid_xlsx_row(max_drop + 1, 70, sheet, True)
    hilight(sheet, list(range(2, 70)), 'f9fff9', [1, 2])
    lattice_xlsx(sheet, 1, 69, 1, sheet.max_column + 1)#上の格子
    for rows in sheet['A71':'K71']:#内容のないセルに入力するため
        for cell in rows:
            cell.fill = PatternFill(fgColor = 'dcdcdc', fill_type = "solid")
    sheet.merge_cells(start_row = 71, end_row = 71, start_column = 1, end_column = 2)
    sheet.merge_cells(start_row = 70, end_row = 70, start_column = 3, end_column = 11)
    lattice_xlsx(sheet, 70, sheet.max_row + 1, 1, sheet.max_column + 1)#下の格子
#Excelに格子をつける
def lattice_xlsx(sheet, start_row, end_row, start_col, end_col):
    side1 =Side(style = 'thin', color = '000000')
    border1 = Border(top = side1, bottom = side1, left = side1, right = side1)
    for r in range(start_row, end_row):
        for col in range(start_col, end_col):
            sheet.cell(row = r, column = col).border = border1
#リストボックスの更新(プロジェクトのみ)
def update_project(py_path):
    files = os.listdir(py_path)
    projects = ['プロジェクト新規作成']
    for file in files:
        if os.path.isdir(os.path.join(py_path, file)):
            projects.append(file)
    files = [s for s in files if not s.startswith('._')]
    return projects
#リストボックスの更新(バイナリのみ)
def update_list(output_dir):
    files = os.listdir(output_dir)
    files = [f for f in files if os.path.isfile(os.path.join(output_dir, f))]
    return files            
#書き出し
def write_out(df, pkl, output_dir, files, output_type):
    for pkl_name in pkl:
        file_name = f'{pkl_name}{output_type}'
        if file_name in files:
            continue
        else:
            df1 = fit_pkl(output_dir, pkl_name, df)
            if output_type == '.xlsx':
                df1.to_excel(f'{pkl_name}{output_type}')
            elif output_type == '.csv':
                df1.to_csv(f'{pkl_name}{output_type}', encoding = 'cp932')
#クエリを開く
def open_query(os_name, book_name, scr_w, scr_h):
    if os_name == 'Windows':
        subprocess.Popen(['start', book_name], shell = True)
        while True:
            try:
                win = pwc.getWindowsWithTitle('query - Excel')[0] 
                break
            except IndexError:
                continue
        win.activate()
        pag.hotkey(('command','win')[os.name=='nt'], 'left')        
    elif os_name == 'Darwin':#Mac用
        subprocess.Popen(['open', book_name])
#クエリを閉じる
def close_excel(excel_name, os_name):
    if os_name == 'Darwin':
        app_name='Excel'
        #cmd = f'osascript -e \'activate application "{app_name}"\''
        #subprocess.call(cmd, shell=True)
        #pag.hotkey('command', 's')
        cmd = f'osascript -e \'quit application "{app_name}"\''
        subprocess.call(cmd, shell=True)
    elif os_name == 'Windows':
        try:
            win = pwc.getWindowsWithTitle(excel_name)[0]
        except IndexError:
            pass
        #win.activate()
        #pag.hotkey('ctrl', 's')
        win.close()
        sleep(2)########
#特定の列を検索
def search_column(sheet, keyword, type, st_row, st_col):
    for r in sheet.iter_rows(max_row = st_row, max_col = st_col):
        for c in r:
            if c.value == keyword:
                cell_nloc=c.coordinate
                cell_nloc_col = re.sub(r"[^a-zA-Z]", "", cell_nloc)
                cell_col_m = openpyxl.utils.column_index_from_string(cell_nloc_col)
                cell_row_n = openpyxl.utils.cell.coordinate_from_string(cell_nloc)[1]
                if type == 'column':
                    return cell_col_m
                elif type == 'row':
                    return cell_row_n
#グラフシートに書き込むpklの絞り込み
def edit_fig_sheet(x):#リストの作成まで
    comment_df = pd.read_excel(x, sheet_name = '抽出')[69:]
    comment_list = comment_df[comment_df['#'].notna()][['#','ファイル名']].values.tolist()
    plot_types = ['書き出し', 'クロス集計',  '相関係数', 'T検定', '箱ひげ図', '散布図','ヒストグラム']
    book = openpyxl.load_workbook(x)
    for plot_type in plot_types:
        enter_fig_sheet(book, comment_list, plot_type, '90ee90')
        if plot_type == '書き出し' or plot_type == '相関係数' or plot_type == 'T検定' or plot_type == 'クロス集計':
            lattice_xlsx(book[plot_type], 1, book[plot_type].max_row + 1, 1, book[plot_type].max_column + 1)
    book.save(x)
#pklの書き込み
def enter_fig_sheet(book, comment_list, plot_type, hi_color):
    sheet = book[plot_type]
    for i in range(len(comment_list)):
        if plot_type == '書き出し' or plot_type == '相関係数' or plot_type == 'T検定'or plot_type == 'クロス集計':
            k = 2
            if plot_type == '相関係数' or plot_type == 'T検定'or plot_type == 'クロス集計':
                end_row = 3
            elif plot_type == '書き出し':
                end_row = 4
        else:
            k = 72
            end_row = 12
        write_list(sheet, [comment_list[i]], i + k, 1)#書き込み
        hilight(sheet, [i + k], hi_color, list(range(1, 3)))#lencolumns
        lattice_xlsx(sheet, i + k, i + k + 1, 1, end_row)
    if plot_type == '相関係数' or plot_type == 'T検定' or plot_type == 'クロス集計':
        hid_xlsx_row(2, 69, sheet, False)
        hid_xlsx_row(len(comment_list) + 2, 69, sheet, True)
#Excelの非表示、表示をする関数(行)
def hid_xlsx_row(start_row, end_row, sheet, dimension_type):
    for i in range(start_row, end_row):
        sheet.row_dimensions[i].hidden = dimension_type
#Excelの非表示、表示をする関数(列)
def hid_xlsx_col(start_col, end_col, sheet, dimension_type):
    for i in range(start_col, end_col):
        col_alphabet = sheet.cell(row=1, column = i).column_letter
        sheet.column_dimensions[col_alphabet].hidden = dimension_type
#フィル受け取り
def auto_fill(fill_keys, facter, fill_type, book_name):
    book = openpyxl.load_workbook(book_name)
    sheet = book['抽出']
    facter_column = search_column(sheet, facter, 'column', 1, sheet.max_column) #+1する
    if fill_type == '一ヶ所':
        fill_keys = ['\n'.join(fill_keys)]
    make_query2(sheet, 0, facter_column, fill_keys, 0, 'Sheet_copy')
    book.save(book_name)
#シートの取得
def get_fig_data(output_dir, book_name, df, project_name, files, sheet_name, frame_x, frame_y): 
    df_fig = pd.read_excel(book_name, sheet_name = sheet_name)
    if sheet_name == '書き出し':
        get_write_out_data(df_fig, df, output_dir, files)
    elif sheet_name == '相関係数' or sheet_name == 'クロス集計':
        get_r_c(df_fig, sheet_name, df, output_dir, book_name)
    elif sheet_name == 'T検定':
        get_t(df_fig, sheet_name, df, output_dir, book_name, frame_x, frame_y)
    else:
        get_plot_data(df_fig, sheet_name, df, output_dir, project_name, frame_x, frame_y)
#書き出しシートの取得
def get_write_out_data(write_out_df, df, output_dir, files):
    to_excel_list = write_out_df[write_out_df['To Excel'] == '*']['ファイル一覧'].to_list()
    to_csv_list = write_out_df[write_out_df['To CSV'] == '*']['ファイル一覧'].to_list()
    write_out(df, to_excel_list, output_dir, files, '.xlsx')
    write_out(df, to_csv_list, output_dir, files, '.csv')
#シート内のデータ取得
def get_plot_data(qx_df, plot_type, df, output_dir, project_name, frame_x, frame_y):
    fig_dir = os.path.join(output_dir, 'fig') #figのフォルダ
    check_dir(fig_dir)    #型のフォルダ作成
    plot_dir = os.path.join(fig_dir, plot_type)
    check_dir(plot_dir)
    df_b_facter = qx_df[:68]#因子のソート
    output_facter = df_b_facter[df_b_facter['書き出し有無'] == '*']['因子'].to_list()#書き出す因子をリストに
    df_facter = df_b_facter[df_b_facter['書き出し有無'] == '*'].drop(columns = '#').set_index('因子')#因子のオプションを含めてDFに
    if df_facter.empty == True:
        return
    df_fig = qx_df[69:].dropna(how = 'all')#ファイルのソート
    df_fig = df_fig.set_axis(list(range(1, len(df_fig.columns) + 1)), axis = 1).set_index(1)
    df_fig_ind = df_fig[2]
    images_paths, images_names, unsave_lists = [], [], []   #セーブしないもののリスト
    save_type = 'unsave'
    for col_num in range(1, len(df_fig.columns)):
        df_pkl_lists = df_fig.iloc[:, col_num]#書き出しリスト作成
        df_fig_data = pd.concat([df_fig_ind, df_pkl_lists], axis = 1)#合成
        if df_fig_data.at['save_fig', col_num + 2] == '*':#セーブするかどうかの判定を先にする
            save_type = 'save'
        df_fig_data = df_fig_data[df_fig_data[col_num + 2] == '*']
        if 'save_fig' in list(map(str, list(df_fig_data.index.values))):#save_fig行が残っていたら削除
            df_fig_data = df_fig_data.drop('save_fig', axis = 0)
        output_pkl = df_fig_data[2].to_list()
        pkl_comments = list(map(str, list(df_fig_data.index.values)))
        pkl_comment = ''.join(pkl_comments)
        if output_pkl == []:#出力するpklの有無を確認
            continue
        if save_type == 'unsave':#削除する画像をリストで保持
            unsave_lists.append(pkl_comment)
        images_path, images_name = make_plot_df(plot_type, output_pkl, output_facter, plot_dir, df, pkl_comments, df_facter, output_dir, pkl_comment, frame_x, frame_y)
        images_paths.extend(images_path)
        images_names.extend(images_name)
        save_type = 'unsave'
    if images_paths == []:
        pass
    else:
        plot_show(images_paths, images_names, project_name)
        for i in range(len(images_paths)):
            image_name = os.path.splitext(os.path.basename(images_paths[i]))[0]
            image_name = image_name.split('_')[1]
            if image_name in unsave_lists:
                image_path = eval(images_paths[i])
                if os.path.exists(image_path):
                    os.remove(image_path)
#フォルダの作成
def make_plot_df(plot_type, pkl_name, facter_name, plot_dir, df, pkl_comments, df_facter, output_dir, pkl_comment, frame_x, frame_y):  
    images_path = []
    images_name = []
    if plot_type == '散布図':#facterをまとめて
        if len(facter_name) == 2:
            fig = plt.figure(figsize = (6.4, 4.8))
            fig = scatter_plot(facter_name, df, fig, pkl_name, output_dir, df_facter)
            facter = ''.join(facter_name)
            image_path = os.path.join(plot_dir, f"{facter}_{pkl_comment}.png")
            plt.savefig(image_path, format="png", dpi=300)
            images_path.append(repr(image_path))
            images_name.append(''.join(facter) + '_' + pkl_comment) 
        else:
            sg.popup_ok('因子数が不適です', title = 'error', location = (frame_x + 100, frame_y + 100))
            pass
    else:#facterごと
        for facter in facter_name:
            image_path = plot_save(plot_type, plot_dir, facter, df, pkl_name, output_dir, pkl_comments, df_facter, pkl_comment)
            images_path.append(repr(image_path))
            images_name.append(''.join(facter) + '_' + pkl_comment)
    plt.close('all')
    return images_path, images_name
#figの作成と保存
def plot_save(plot_type, plot_dir, facter, df, pkl_name, output_dir, pkl_comments, df_facter, pkl_comment):
    fig = plt.figure(figsize = (6.4, 4.8))
    if plot_type == '箱ひげ図':
        fig = box_plot(facter, df, fig, pkl_name, output_dir, pkl_comments, df_facter)
    elif plot_type == 'ヒストグラム':
        fig = his_plot(facter, df, fig, pkl_name, output_dir, pkl_comments, df_facter)
    image_path = os.path.join(plot_dir, f"{facter}_{pkl_comment}.png")
    plt.savefig(image_path, format="png", dpi=300)
    return image_path
#箱ひげ図作成(因子、全体DF、figの型、使用するpkl、OutputDir、コメント)
def box_plot(key, df, fig, pkl_name, output_dir, pkl_comments, df_facter):
    plt_lists = []  
    key = ''.join(key)
    outlier_type = df_facter.loc[key, '外れ値']
    ver_t = df_facter.loc[key, '向き']
    ver_type = True ## muki
    if ver_t == '*':
        ver_type = False
    for pkl in pkl_name:
        df_pkl = fit_pkl(output_dir, pkl, df)
        df_pkl = list_fresh(list(df_pkl[key]))
        plt_lists.append(df_pkl)
    plt.title(key, fontsize=20,)
    check_scale('y', ['最大値', '最小値', '目盛間隔'], df_facter, key)
    if outlier_type == '*':
        plt.boxplot(plt_lists, labels = pkl_comments, sym = '', vert = ver_type)
    else:
        plt.boxplot(plt_lists, labels = pkl_comments, vert = ver_type)
    return fig
#ヒストグラム作成(因子、全体DF、figの型、使用するpkl、OutputDir、コメント)
def his_plot(key, df, fig, pkl_name, output_dir, pkl_comments, df_facter):
    plt_lists = []
    bins_list = check_scale('x', ['x最大値', 'x最小値', 'x目盛間隔', '階級幅'], df_facter, key)
    check_scale('y', ['y最大値', 'y最小値', 'y目盛間隔'], df_facter, key)
    for i in range(len(pkl_name)):
        df_pkl = fit_pkl(output_dir, pkl_name[i], df)   
        key = ''.join(key)
        df_pkl = list_fresh(list(df_pkl[key]))
        plt_lists.append(df_pkl)
        try:
            bins_list
        except NameError:
            plt.hist(df_pkl, alpha = 0.5, label = pkl_comments[i])
        else:
            plt.hist(df_pkl, alpha = 0.5, label = pkl_comments[i], bins = bins_list)
    plt.title(key, fontsize=20)
    plt.legend()
    return fig
#散布図作成(因子、全体DF、figの型、使用するpkl、OutputDir、コメント)
def scatter_plot(keys, df, fig, pkl_name, output_dir, df_facter):
    pkl = pkl_name[0]
    df_pkl = fit_pkl(output_dir, pkl, df)
    df_pkl.plot.scatter(x = keys[0], y = keys[1])
    df_p = df_pkl.dropna()
    x_r = list_fresh(list(df_p[keys[0]]))
    y_r = list_fresh(list(df_p[keys[1]]))
    p = np.polyfit(x_r, y_r, 1)
    f = np.poly1d(p)
    a = np.corrcoef(x_r, y_r)[0][1]
    check_scale('y', ['最大値', '最小値', '目盛間隔'], df_facter, keys[1])
    check_scale('x', ['最大値', '最小値', '目盛間隔'], df_facter, keys[0])
    plt.plot(x_r, f(x_r), color = "r", label=round(a, 3))
    plt.legend()
    return fig
#画像をリサイズ、bytes形式で返す
def load_bytesImg(img_path, size_ratio=1):
    img = Image.open(img_path)
    resize_img = img.resize((int(img.width*size_ratio), int(img.height*size_ratio)))
    output = io.BytesIO()
    resize_img.save(output, format="png")
    return output.getvalue()
#T検定
def get_t(df_fig, sheet_name, pre_df, output_dir, book_name, frame_x, frame_y):
    df_columns = list(pre_df.columns)
    int_columns = []
    book = openpyxl.load_workbook(book_name)
    sheet = book[sheet_name]   
    lattice_st_row = sheet.max_row + 1
    hid_xlsx_row(100, lattice_st_row, sheet, True)#すでにある表を非表示
    #クエリの読み込み
    df_pkl = df_fig[:69]
    pkls = df_pkl[df_pkl['書き出し有無'] == '*']['ファイル一覧'].to_list()
    comments = df_pkl[df_pkl['書き出し有無'] == '*']['コメント'].to_list()
    if len(pkls) == 0:
        return
    elif len(pkls) != 2:
        sg.popup_ok('入力ミスがあります', title = 'error', location = (frame_x + 100, frame_y + 100))
        return    
    df_type = df_fig[69:97].dropna(how = 'all').drop('コメント', axis = 1).set_index('ファイル一覧')
    side_type = 'two-sided'
    test_type = False
    if df_type.at['greatar', '書き出し有無'] == '*':
        side_type = 'greatar'
    elif df_type.at['less', '書き出し有無'] == '*':
        side_type = 'less'
    elif df_type.at['両側', '書き出し有無'] == '*':
        side_type = 'two-sided'
    if df_type.at['有り', '書き出し有無'] == '*':
        test_type = 'rel'
    elif df_type.at['無し', '書き出し有無'] == '*':
        test_type = False
    for i in range(len(df_columns)):#文字のドロップ
        if pre_df[df_columns[i]].dtypes == 'O':
            pass
        else:
            int_columns.append(df_columns[i])
    df1 = fit_pkl(output_dir, pkls[0], pre_df)
    df2 = fit_pkl(output_dir, pkls[1], pre_df)
    t_lists = []
    for i in range(len(int_columns)):
        df1_facter = list(df1[int_columns[i]])
        df2_facter = list(df2[int_columns[i]])
        if test_type == 'rel':
            stats.ttest_rel(df2_facter, df1_facter)
        else:
            t_list = list(stats.ttest_ind(df2_facter, df1_facter, nan_policy = 'omit',equal_var = test_type, alternative = side_type))
            t_lists.append((list(map(round, t_list, [3]*len(t_list)))))
            t_lists[i].extend([len(df1_facter), len(df2_facter)])
    t_lists.insert(0, ['', 'T値', 'p値', f'{comments[0]}:n数', f'{comments[1]}:n数'])
    for i in range(len(int_columns)):
        t_lists[i + 1].insert(0, int_columns[i])
    for i in range(len(t_lists)):
        if i == 0:
            k = sheet.max_row + 1
        write_list(sheet, [t_lists[i]], sheet.max_row + 1, 1)
    hilight(sheet, [k], '66cdaa', list(range(1, 4)))
    hilight(sheet, list(range(k + 1, k + len(t_lists))), '90ee90', [1])
    for row in sheet.iter_rows(min_row = lattice_st_row + 1, min_col = 3):
        for cell in row:
            if cell.value is None:
                pass
            elif abs(float(cell.value)) < 0.05:
                cell.fill = PatternFill(fgColor='FFFF00',bgColor="FFFF00", fill_type = "solid")
    lattice_xlsx(sheet, 70, sheet.max_row + 1, 1, sheet.max_column + 1)
    book.save(book_name)
    return
#フォルダの有無を確認なければ作る
def check_dir(fig_dir):
    try:
        os.makedirs(fig_dir)
    except FileExistsError:
        pass
#クロス、相関係数分岐
def get_r_c(df_fig, sheet_name, df, output_dir, book_name):
    book = openpyxl.load_workbook(book_name)
    sheet = book[sheet_name]
    sheet_max_row = sheet.max_row + 1
    hid_xlsx_row(70, sheet_max_row, sheet, True)#すでにある表を非表示
    #クエリの読み込み
    pkls = df_fig[df_fig['書き出し有無'] == '*']['ファイル一覧'].to_list()
    df_columns = list(df.columns)
    for pkl in pkls:
        pre_df = fit_pkl(output_dir, pkl, df)
        if sheet_name == 'クロス集計':
            get_cross(pre_df, df_columns, sheet, pkl)
        elif sheet_name == '相関係数':
            get_r(pre_df, df_columns, sheet, pkl)
    lattice_xlsx(sheet, sheet_max_row, sheet.max_row + 1, 1, sheet.max_column + 1)
    book.save(book_name)
#クロス集計表
def get_cross(pre_df, df_columns, sheet, pkl):
    cross_col_lists = [] #グラフシートに書き込む因子のリスト
    for i in range(0,len(df_columns)): ##中身の型確認
        drop_list = list(dict.fromkeys(pre_df[f'{df_columns[i]}'].to_list()))
        if pre_df[df_columns[i]].dtypes == 'float':
            if len(drop_list) < 10:
                cross_col_lists.append(df_columns[i])
        elif len(drop_list) < 68:
            cross_col_lists.append(df_columns[i])       
    if len(cross_col_lists) < 2:
        return
    cross_pairs = []
    for pair in itertools.combinations(cross_col_lists, 2):
        cross_pairs.append(list(pair))
    for cross_pair in cross_pairs:
        cross_tab = pd.crosstab(pre_df[cross_pair[0]], pre_df[cross_pair[1]])
        cross_tab_col = cross_tab.columns.to_list()
        cross_tab_ind = cross_tab.index.to_list()
        cross_tab_1 = cross_tab.values.tolist()
        cross_tab_col.insert(0, pkl)
        for i in range(len(cross_tab_ind)):
            cross_tab_1[i].insert(0, cross_tab_ind[i])
        cross_tab_1.insert(0, cross_tab_col)
        k = sheet.max_row + 1
        write_list(sheet, cross_tab_1, sheet.max_row + 1, 1)
        hilight(sheet, [k], '66cdaa', list(range(len(cross_tab_1[0]) + 1)))
        hilight(sheet, list(range(k + 1, k + len(cross_tab_1))), '90ee90', [1])
#相関係数
def get_r(pre_df, df_columns, sheet, pkl):
    int_columns = []
    for i in range(len(df_columns)):#文字のドロップ
        if pre_df[df_columns[i]].dtypes == 'float':
            int_columns.append(df_columns[i])
        else:
            pre_df = pre_df.drop(df_columns[i], axis = 1)
    df_corr = pre_df.corr().round(3).values.tolist()
    df_corr.insert(0, int_columns)#先頭にカラム名の列を追加
    int_columns.insert(0, pkl)
    for i in range (len(int_columns) - 1):#各行の先頭にカラム名をつける
        df_corr[i + 1].insert(0, int_columns[i + 1])
    lattice_st_row = sheet.max_row + 1
    for i in range(len(df_corr)):
        if i == 0:
            k = sheet.max_row + 1
        write_list(sheet, [df_corr[i]], sheet.max_row + 1, 1)
    hilight(sheet, [k], '66cdaa', list(range(1, len(df_corr[0]) + 1)))
    hilight(sheet, list(range(k + 1, k + len(df_corr))), '90ee90', [1])
    for row in sheet.iter_rows(min_row = lattice_st_row + 1, min_col = 2):
        for cell in row:
            if cell.value is None:
                pass
            elif abs(float(cell.value)) > 0.4 and abs(float(cell.value)) != 1:
                #該当セルに色付け
                cell.fill = PatternFill(fgColor='FFFF00',bgColor="FFFF00", fill_type = "solid")
            elif abs(float(cell.value)) == 1:
                cell.fill = PatternFill(fgColor='808080', fill_type = "solid")
#pklの適応
def fit_pkl(output_dir, pkl, df):
    pkl_path = os.path.join(output_dir, pkl)
    with open(f'{pkl_path}', 'rb') as p:
        index_list = pickle.load(p)
    pre_df = df.loc[index_list]  ##pklの適応
    return pre_df
#リストを数字にして、nanを削除
def list_fresh(pre_list):
    x = [float(s) for s in pre_list]
    mat_list = [k for k in x if math.isnan(k) == False]
    return mat_list
#数字であることを確認
def check_float(x):
    if str(x) != 'nan':
        try:
            x = float(x)
        except ValueError:
            return x
        else:
            return x
#軸パラメータの確認
def check_scale(axis_type, params, df_facter, key):
    params_list = []
    for param in params:
        params_list.append(check_float(df_facter.loc[key, param]))
    if type(params_list[0]) == float and type(params_list[1]) == float:
        if axis_type == 'y':
            if type(params_list[2]) == float:
                plt.yticks(np.arange(params_list[1], params_list[0] + 1, params_list[2]))        
            plt.ylim(params_list[1], params_list[0])
        elif axis_type == 'x':
            if type(params_list[2]) == float:
                plt.xticks(np.arange(params_list[1], params_list[0] + 1, params_list[2]))        
            plt.xlim(params_list[1], params_list[0])
        try:
            params_list[3]
        except NameError:
            return
        else:
            bins_list = range(int(params_list[1]), int(params_list[0]), int(params_list[3]))
            return bins_list
#クエリ作成　抽出
def make_query_dxt(sheet, df_columns, df):
    len_column = search_column(sheet, 'end_t', 'column', 1, sheet.max_column) 
    max_drop = 0
    float_col_lists = [] #グラフシートに書き込む因子のリスト
    for i in range(0,len(df_columns)): ##中身の型確認
        sheet.cell(1, len_column + i + 1, value = df_columns[i])
        drop_list = list(dict.fromkeys(df[f'{df_columns[i]}'].to_list()))
        if df[df_columns[i]].dtypes == 'float':
            float_col_lists.append(df_columns[i])
            if len(drop_list) < 10:
                max_drop = make_query2(sheet, 3, len_column + i + 1, drop_list, max_drop)
        elif len(drop_list) < 68:
            max_drop = make_query2(sheet, 3, len_column + i + 1, drop_list, max_drop)
    sheet.cell(1, len_column + len(df_columns) + 1, value = 'end_f')
    hid_xlsx_col(len_column + len(df_columns) + 1, len_column + len(df_columns) + 2, sheet, True)#end_t
    hid_xlsx_row(3, 70, sheet, True)#ドロップリスト
    hid_xlsx_col(5, len_column + 1, sheet, True)#end_f
    max_column = sheet.max_column + 1
    hilight_lens = [list(range(len_column + 1, max_column)), list(range(1, max_column)), list(range(1, max_column))]
    hilight_rows = [[1], [2], list(range(3, 70))]
    hilight_colors = ['90ee90', 'dcdcdc', 'f9fff9']
    for i in range(len(hilight_lens)):
        hilight(sheet, hilight_rows[i], hilight_colors[i], hilight_lens[i]) 
    lattice_xlsx(sheet, 1, 71, 1, max_column)
    sheet.merge_cells(start_row = 70, end_row = 70, start_column = 1, end_column = max_column-1)
    return float_col_lists, len_column 

main()

Mac OS Version is 13.3.1 and patch enabled so applying the patch


In [3]:
# #テスト
# pre_df = pd.read_csv('/Volumes/USB DISK/sort_pg/A/abc.csv', encoding = 'cp932', index_col=0)
# book_name = '/Volumes/USB DISK/sort_pg/A/query.xlsx'
# sheet_name = '相関係数'
# df_fig = pd.read_excel(book_name, sheet_name = sheet_name)
# book = openpyxl.load_workbook(book_name)
# df_columns = list(pre_df.columns)
# output_dir = '/Volumes/USB DISK/sort_pg/A/output'


ワークシートの行数と列数  
    1,048,576 行、16,384 列  

In [4]:
seaborn
.bat
.commamd


SyntaxError: invalid syntax (3313568527.py, line 2)

In [ ]:
# cd `dirname $0`
# python renkei_gch.py
# echo off